# Problem 26
A unit fraction contains 1 in the numerator. The decimal representation of the unit fractions with denominators 2 to 10 are given:

1/2	= 	0.5
1/3	= 	0.(3)
1/4	= 	0.25
1/5	= 	0.2
1/6	= 	0.1(6)
1/7	= 	0.(142857)
1/8	= 	0.125
1/9	= 	0.(1)
1/10	= 	0.1
Where 0.1(6) means 0.166666..., and has a 1-digit recurring cycle. It can be seen that 1/7 has a 6-digit recurring cycle.

Find the value of d < 1000 for which 1/d contains the longest recurring cycle in its decimal fraction part.

---

Usefull pages: https://en.wikipedia.org/wiki/Repeating_decimal and https://thestarman.pcministry.com/math/rec/RepeatDec.htm

From this we learn that we dont care about 1/(2^n * val) and 1/(5^n * val), since they will just result in times .2 and times .5, and will not have an effect on the recurring cycle. Furthermore we note that often primes create such a recurrance cycle. However, (1/7 * 1/7) = 1/49 creates a different recurrance cycle, thus we cannot only take the primes. We also have to take into account if the number factorized into its prime factors (except for 2 and 5).

For the cycle length we use a trick that when you devide the denometnator of the fraction by 9, 99, 999, ... where the number of nines represents the length of the cycle, results in an integer value. This would be most easily implemented using a module operator. However then you get into floating point division accuracy (+-7 decimal digits), thus we implement it as an integer multiplication for which python seems to have an infinite (rather computer limited), precision.

In [10]:
# An example using 1/7
denominator = 7
numerator = 0
for i in range(1, 9):
    numerator += 9*10**i
    print(i+1, (numerator / denominator)%1)
# for length 7 there is 0 leftover, hence this is the length of the cycle.

2 0.8571428571428577
3 0.4285714285714164
4 0.14285714285711038
5 0.285714285713766
6 0.714285714289872
7 0.0
8 0.8571428563445807
9 0.4285714328289032


In [11]:
import itertools

# To factorize a number into its prime factor
def get_prime_factors(n):
    for i in itertools.chain([2], itertools.count(3, 2)):
        if n <= 1:
            break
        while n % i == 0:
            n //= i
            yield i

# The 999 algorithm to get the length of the cycle
def get_repeating_decimal_length(number):
    length = 0
    numerator = 0
    remainder = 3.14159265359

    i = 0
    while remainder != 0:  
        numerator += 9 * 10**i
        denominator = 1*number

        factor = numerator//denominator
        check_val = factor * denominator
        
        if check_val == numerator:
            remainder = 0
        else:
            pass
        #remainder = (numerator/denominator)%1

        length += 1
        i += 1
        
    return length

# A different range implementation such that we skip multiples of 2 and 5.
def varied_step_range(start,stop,stepiter):
    step = itertools.cycle(stepiter)
    while start < stop:
        yield start
        start += next(step)

In [19]:
%%timeit -n1 -r1
from functools import reduce
from collections import Counter

max_cycle_val = 0
max_cycle_length = 0

for i in varied_step_range(3, 1000, [4,2,2,2]):
    factor_count_dict = Counter(list(get_prime_factors(i)))
    divisor = 1
    for key in factor_count_dict.keys():
        divisor *= key**(factor_count_dict[key])
        
    repeating_decimal_length = get_repeating_decimal_length(divisor)
    
    if repeating_decimal_length > max_cycle_length:
        max_cycle_length = 1*repeating_decimal_length
        max_cycle_val = i
        
print(max_cycle_val, max_cycle_length)

983 982
101 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


100 ms is ok by me. Could probably be vastly improved.

An algorithm just going over the primes. I do not think that covers all cases:

In [122]:
%%timeit -n1 -r1
primes = list(filter(lambda x: is_prime(x), varied_step_range(3, 1000, [4,2,2,2])))

max_cycle_val = 0
max_cycle_length = 0

for prime in primes:
    repeating_decimal_length = get_repeating_decimal_length(prime)
    
    if repeating_decimal_length > max_cycle_length:
        max_cycle_length = 1*repeating_decimal_length
        max_cycle_val = prime

#prime_repeating_length = dict(prime_repeating_length)

72.8 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
